In [1]:
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as fn
from pyspark.sql import SparkSession
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
spark = (SparkSession.builder
         .config("spark.jars","""/home/jovyan/jars/aws-java-sdk-core-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-dynamodb-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-s3-1.11.534.jar,
                                 /home/jovyan/jars/hadoop-aws-3.2.2.jar""")
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.access.key", "aulafia")
         .config("spark.hadoop.fs.s3a.secret.key", "aulafia@123")
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
         .getOrCreate()
        )

In [3]:
df_estado = spark.read.format('parquet').load('s3a://bronze/ibge/estado')
df_estado.dtypes

[('id', 'string'),
 ('nome', 'string'),
 ('regiao', 'struct<nome:string,sigla:string,id:string>'),
 ('sigla', 'string')]

In [4]:
df_estado.show(12, False)

+---+------------------+---------------------+-----+
|id |nome              |regiao               |sigla|
+---+------------------+---------------------+-----+
|27 |Alagoas           |{Nordeste, NE, 2}    |AL   |
|28 |Sergipe           |{Nordeste, NE, 2}    |SE   |
|29 |Bahia             |{Nordeste, NE, 2}    |BA   |
|31 |Minas Gerais      |{Sudeste, SE, 3}     |MG   |
|32 |Espírito Santo    |{Sudeste, SE, 3}     |ES   |
|33 |Rio de Janeiro    |{Sudeste, SE, 3}     |RJ   |
|35 |São Paulo         |{Sudeste, SE, 3}     |SP   |
|41 |Paraná            |{Sul, S, 4}          |PR   |
|42 |Santa Catarina    |{Sul, S, 4}          |SC   |
|43 |Rio Grande do Sul |{Sul, S, 4}          |RS   |
|50 |Mato Grosso do Sul|{Centro-Oeste, CO, 5}|MS   |
|51 |Mato Grosso       |{Centro-Oeste, CO, 5}|MT   |
+---+------------------+---------------------+-----+
only showing top 12 rows



In [5]:
df_estado_campos = df_estado.select(fn.col("id").cast('int').alias("estado_id"), 
                                    fn.col("nome").alias("estado_nome"), 
                                    fn.col("sigla").alias("estado_sigla"), 
                                    fn.col("regiao.id").cast('int').alias("regiao_id"),
                                    fn.col("regiao.nome").alias("regiao_nome"),
                                    fn.col("regiao.sigla").alias("regiao_sigla")
                                   )

In [6]:
df_estado_campos.show(12, False)

+---------+------------------+------------+---------+------------+------------+
|estado_id|estado_nome       |estado_sigla|regiao_id|regiao_nome |regiao_sigla|
+---------+------------------+------------+---------+------------+------------+
|27       |Alagoas           |AL          |2        |Nordeste    |NE          |
|28       |Sergipe           |SE          |2        |Nordeste    |NE          |
|29       |Bahia             |BA          |2        |Nordeste    |NE          |
|31       |Minas Gerais      |MG          |3        |Sudeste     |SE          |
|32       |Espírito Santo    |ES          |3        |Sudeste     |SE          |
|33       |Rio de Janeiro    |RJ          |3        |Sudeste     |SE          |
|35       |São Paulo         |SP          |3        |Sudeste     |SE          |
|41       |Paraná            |PR          |4        |Sul         |S           |
|42       |Santa Catarina    |SC          |4        |Sul         |S           |
|43       |Rio Grande do Sul |RS        

In [7]:
(df_estado_campos
 .write
 .format('parquet')
 .mode('overwrite')
 .save('s3a://silver/dado_geografico/estado')
 )

In [8]:
pandasDF = df_estado_campos.toPandas()

In [11]:
pandasDF.to_excel('api_ibge_estados_export.xlsx', sheet_name='estados', index = False)